In [1]:
import numpy as np
import mesh_to_sdf
import trimesh
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import tensorflow as tf
import mcubes
import open3d as o3d
from Subdivision.Datastructures import Octree, OctreeNode
import yaml
from tqdm import tqdm
import torch
import sys
sys.path.insert(0, '../')
from nsdf.models import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("../3DModels/monkj.obj")
bounds = mesh.get_axis_aligned_bounding_box()
extent = np.max(bounds.get_extent(), axis=-1)
mesh.scale(1. / extent, center=bounds.get_center())

TriangleMesh with 3229 points and 1149 triangles.

In [3]:
verts = np.array(mesh.vertices)
print(verts.min(), verts.max())

-0.49999999999999994 0.49999999999999994


In [4]:
N = 250000
pcd = mesh.sample_points_uniformly(N)
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
np.array(pcd.points).max()
#o3d.visualization.draw_geometries([pcd])

0.4992025904306532

In [5]:
def subdivide(points, depth):
    return True

def drop(points, depth):
    MEAN = 992058.9606299213
    STD = 706475.1765405646
    volume = (1. / 2**depth) ** 3
    density = points.shape[0] / volume
    z = (density - MEAN) / STD
    if z < -1.32:
        return True
    return False

points = np.random.random((300, 3)) - 0.5
print(points.min(), points.max())

octree = Octree(origin = np.array([-.5, -.5, -.5]),
                extents = np.array([1., 1., 1.]),
                points = np.array(pcd.points),
                max_depth = 3,
                subdivide = subdivide,
                drop = drop)
octree.visualize(show_boxes=False, show_coordinate_frames=True, show_points=True)
#print(octree.root_node.get_child_bounding_box(0))
#print(octree.get_debug_string())
densities = [node.points.shape[0] / node.extents[0]**3 for node in octree.get_all_leaf_nodes()]
densities.sort()
densities = np.array(densities)
print(densities.min(), densities.max(), densities.mean(), densities.std())
z_scores = (densities - densities.mean()) / densities.std()
print(z_scores.min(), z_scores.max())
#densities


-0.49970825481711567 0.49913057182560117
60864.0 3182080.0 1145343.4128440367 647489.5469045822
-1.674898719258941 3.145589912443959


In [6]:
t_mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

scene = o3d.t.geometry.RaycastingScene()
_ = scene.add_triangles(t_mesh)


min_bound = t_mesh.vertex['positions'].min(0).numpy()
max_bound = t_mesh.vertex['positions'].max(0).numpy()

N = 10_000_000
SURFACE_TO_UNIFORM_RATIO = 2
PURTURB_AMOUNT = 0.005

uniform_sample = np.random.uniform(low=min_bound, high=max_bound,
                                 size=[int(N / SURFACE_TO_UNIFORM_RATIO), 3]).astype(np.float32)

surface_purturbed_points = np.array(mesh.sample_points_uniformly(N - uniform_sample.shape[0]).points).astype(np.float32)
surface_purturbed_points += np.random.normal(loc=0, scale=PURTURB_AMOUNT, size=surface_purturbed_points.shape).astype(np.float32)

points = np.concatenate([surface_purturbed_points, uniform_sample])
np.random.shuffle(points)
signed_distance = scene.compute_signed_distance(points).numpy()
print(points.shape, signed_distance.shape)

(10000000, 3) (10000000,)


In [7]:
np.unique(octree.root_node.occupancy(points), return_counts=True)

(array([False,  True]), array([   1460, 9998540], dtype=int64))

In [8]:
inside_points = points[signed_distance <= 0]
inside_points.shape

(3646332, 3)

In [9]:
inside_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(inside_points))
o3d.visualization.draw_geometries([inside_pcd])

In [10]:
voxels = []
leaf_nodes = octree.get_all_leaf_nodes()
for node in leaf_nodes:
    inside = node.occupancy(points)
    node_points = points[inside]
    dists = signed_distance[inside]
    voxels.append((OctreeNode(node.origin,
                             node.extents,
                             node_points,
                             node.path),
                             dists))

In [11]:
len(voxels)

109

In [12]:
index = 1#np.random.randint(0, len(voxels)-1)
points_shape = leaf_nodes[index].points
points_sampled = voxels[index][0].points
pcd_shape = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points_shape))
pcd_sample = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points_sampled))

pcd.paint_uniform_color(np.array([0, 0, 1]))


red = np.tile(np.array([1, 0, 0]), (points_sampled.shape[0], 1))
green = np.tile(np.array([0, 1, 0]), (points_sampled.shape[0], 1))

colors = np.where(np.expand_dims(voxels[index][1] < 0, -1), green, red)

pcd_sample.colors = o3d.utility.Vector3dVector(colors)
geometries = []
#geometries += [pcd_shape]
geometries += [pcd]
geometries += [pcd_sample]

o3d.visualization.draw_geometries(geometries)

In [13]:
# def get_model(size, embedding_size, hidden_layers):
#     device = torch.device("cpu:0")
#     model = SDFDNN(
#         size,
#         embedding_size,
#         hidden_layers=hidden_layers,
#         activation_type='relu',
#     ).to(device)

#     return device, model

# def train_model(model, device, X, y, epochs=100, batch_size=512, shuffle_after_each_epoch=False):
#     def get_batch(index):
#         index %= (X.shape[0] + batch_size - 1) // batch_size
#         start = min(index*batch_size, X.shape[0])
#         end = min((index+1)*batch_size, X.shape[0])
#         return X[start:end], y[start:end]

#     def shuffle_data(X, y):
#         index = np.arange(X.shape[0])
#         np.random.shuffle(index)
#         X = X[index]
#         y = y[index]
    
#     loss_fn = torch.nn.MSELoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)
#     scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

#     for _ in tqdm(range(1, epochs+1)):
#         for i in range((X.shape[0] + batch_size - 1) // batch_size):
#             inputs, targets = get_batch(i + 1)
#             inputs = torch.Tensor(inputs).to(device)
#             targets = torch.Tensor(targets).to(device).unsqueeze(1)

#             predictions = model(inputs)
#             loss = torch.mean(loss_fn(predictions, targets))

#             # Backpropagation.
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # Update LR scheduler.
#             if i % (epochs // 3) == 0:
#                 scheduler.step()
            
#             if shuffle_after_each_epoch:
#                 shuffle_data(X, y)
#     print(f"Finished Training Model...MSE={loss.item():.6f}")


In [14]:
class NeuralSDFModel(tf.keras.Model):
    def __init__(self, fourier_features_size, hidden_layer_size, hidden_layer_count):
        super(NeuralSDFModel, self).__init__()
        self.fourier_features_mapping = tf.random.normal((3, fourier_features_size))
        self.fourier_alpha = tf.Variable(1.0)
        self.hidden_layers = [tf.keras.layers.Dense(hidden_layer_size, activation="relu") for _ in range(hidden_layer_count)]
        self.output_layer = tf.keras.layers.Dense(1)
    
    def call(self, x):
        x = tf.matmul(2 * tf.constant(np.pi) * x, self.fourier_features_mapping * self.fourier_alpha)
        for d in self.hidden_layers:
            x = d(x)
        return self.output_layer(x)

In [15]:
EPOCHS = 5000
RESCALE = True
CENTER_AT_ORIGIN = True
models = []

class TQDMCallback(tf.keras.callbacks.Callback):
    def __init__(self, pbar):
        self.pbar = pbar
        self.total_loss = 0
        self.epoch_count = 0
    
    def on_epoch_end(self, epoch, logs=None):
        self.pbar.update(1)
        if self.epoch_count > 0:
            self.pbar.set_postfix({'loss': self.total_loss / self.epoch_count})
        self.total_loss += logs['loss']
        self.epoch_count += 1


with tqdm(total=EPOCHS*len(voxels)) as pbar:
    def increment():
        pbar.update(1)
    for i, (node, dists) in enumerate(voxels):
        points = node.points
        if RESCALE:
            points = node.points
            points -= node.origin
            points /= node.extents
            dists /= node.extents.max()
            if CENTER_AT_ORIGIN:
                points -= 0.5
        
        #print(f"Training model #{i+1} out of {len(voxels)}...")
        model = NeuralSDFModel(8, 8, 2)
        model.compile(optimizer='adam', loss='MSE', metrics=['MSE'])
        model.fit(points, dists, epochs=EPOCHS, batch_size=4096, verbose=0, callbacks=[TQDMCallback(pbar)])
        #print(model.evaluate(points, dists))
        #train_model(model, device, points, dists, EPOCHS, 512, shuffle_after_each_epoch=False)
        
        models.append(model)

100%|██████████| 545000/545000 [2:23:53<00:00, 63.13it/s, loss=0.000506]  


In [20]:
points_pred = np.random.random(size=(10000_000, 3)) - 0.5
points_shape = np.zeros((0, 3))
for i, (node, _) in enumerate(tqdm(voxels)):
    points_inside = points_pred[node.occupancy(points_pred)]
    if RESCALE:
        points -= node.origin
        points /= node.extents
        if CENTER_AT_ORIGIN:
            points -= 0.5 
    dists = models[i].predict(points_inside).reshape(-1)
    pp = points_inside[dists < 0]
    points_shape = np.vstack([points_shape, pp])

100%|██████████| 109/109 [01:11<00:00,  1.51it/s]


In [21]:
pred_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points_shape))
o3d.visualization.draw_geometries([pred_pcd])